# Help on Doing Graphs Uniformly for the Website

### 0 - Import Necessary Libraries

In [1]:
import base64
from IPython.display import display, HTML
import os
import json
import plotly.graph_objects as go
import plotly.io as pio

import numpy as np
import pandas as pd


### 1 - Font Importation and Color Palette


In [141]:
### Imports the font in your current notebook

current_dir = os.getcwd()
font_path = os.path.join(current_dir, "src/utils/JetBrainsMono-Regular.otf")

# Read and encode the font file
with open(font_path, "rb") as font_file:
    encoded_font = base64.b64encode(font_file.read()).decode()

font_css = f"""
<style>
@font-face {{
    font-family: 'JetBrains Mono';
    src: url(data:font/otf;base64,{encoded_font});
}}
</style>
"""
display(HTML(font_css))


font_css_playfair = """
<style>
@font-face {
    font-family: 'Playfair Display';
    src: url('https://fonts.gstatic.com/s/playfairdisplay/v13/pxiDypQpkdL4i7D9xfnt8qJyyXopCYe0Obe6jP9UXQ.ttf') format('truetype');
    font-weight: 900;
    font-style: italic;
}
</style>
"""
display(HTML(font_css_playfair))

### Color palette
yellow_palette=["#FFEE00","#D5C923","#AAA239","#807B40","#555339","#33322B"] # goes from bright yellow to dark yellow

### 2 - Graph Example - Pie Chart

In [6]:
# Get all data for this specific graph

from src.data.utils import get_franchise_movies, get_franchise_data, get_inflation_rate, get_movie, get_genre_counts

inflation_rate_df=get_inflation_rate()
movies_df_raw = pd.read_csv('data/movie_metadata_with_tmdb.csv')
franchise_df=get_franchise_movies(movies_df_raw,inflation_rate_df,'data/missing_dates_manualsearch.csv')
movies_df=get_movie(movies_df_raw,inflation_rate_df)
movies_no_franchise_df = movies_df[movies_df['collection_id'].isna()].reset_index(drop=True)
franchise_data_df = get_franchise_data(franchise_df)

proportion = len(franchise_df) / len(movies_df)
proportions = pd.DataFrame({'Proportion': [proportion, 1 - proportion]}, index=['In franchise', 'Not in franchise'])


In [169]:
# Create the general plot

fig = go.Figure(go.Pie(
    labels=proportions.index,
    values=proportions['Proportion'], 

    marker_line=dict(color='black', width=1),
    marker=dict(colors=["#2471ff", "rgba(0, 0, 0, 0)"]),
    pull=[0, 0.1],               #pull is the distance between the slices (to move them apart)
    textinfo='percent',          #text is what is displayed inside the plot
    textfont=dict(size=10),  
    hoverinfo='label+percent',   #hover is what is displayed when you hover over the plot
    hoverlabel=dict(font=dict(family='Jetbrains Mono', size=8)),
    hole=0.4,                    # create the "donut" effect
))

# Modify the layout

fig.update_layout(
    
    #title="Movies in a franchise <br><span style='font-size:12px;'>among the data set</span>",

    font=dict(
        size=30,
        family="JetBrains Mono",
        color="black",
    ),
    legend=dict(
        font=dict(size=10),
        xanchor='left',
        x=0.80,
        yanchor='top',
        y=0.8,
    ),
    paper_bgcolor='rgba(0,0,0,0)', 
    plot_bgcolor='rgba(0,0,0,0)',
    autosize=True,
)
fig.add_annotation(
    x=0.35,  # X-coordinate near the slice
    y=1,  # Y-coordinate near the slice
    xanchor='right',
    text="Our 5163 movies  ",  # Text to display
    showarrow=False,  # Show arrow pointing to the slice
    # arrowhead=7,  # Adjust arrow head
    # ax=-40,  # Arrow head position
    # ay=-20,  # Arrow head position
    # arrowcolor="#2471ff",
    font=dict(family='JetBrains Mono' , size=18,  color="#2471ff")  # Customize font
)

# Add curved arrow
fig.add_shape(
    type="path",
    path="M 0.34,0.97 Q 0.38,1 0.41,0.95",  # SVG path for a curved arrow
    line=dict(color="#2471ff", width=2),  # Arrow line style
    xref="paper",  # Path coordinates relative to the chart
    yref="paper"
)

# Add arrowhead manually as a small triangle
fig.add_shape(
    type="path",
    path="M 0.400,0.937 L 0.411,0.964 L 0.415,0.93 Z",  # Triangle for the arrowhead
    fillcolor="#2471ff",
    line=dict(color="#2471ff"),
    xref="paper",
    yref="paper")

fig.show() # uncomment to show the plot in the notebook

### 3 - Save the plot to the website.

In [166]:
##---TO MODIFY---##
website_dir = "/Users/clementloyer/GitHub/ada-website.github.io" ### Change to your 'ada_website' directory

graph_name = "franchise_pie" ### Assign graph name
##---------------##

graph_json_path = website_dir + "/_includes/graphs/" + graph_name + ".json"
graph_html_path = website_dir + "/_includes/graphs/" + graph_name + ".html"
# Save the graph as JSON (this only includes the data and layout)
pio.write_json(fig, graph_json_path)

with open(graph_json_path, 'r') as json_file:
    graph_data = json.load(json_file)

# Create the minimal HTML structure with Plotly CDN
html_content = f"""
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Plotly Graph</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <div id="{graph_name}" style="width: 100%; height: 500px;"></div>
    <script>
        var graphData = {json.dumps(graph_data)};
        Plotly.newPlot('{graph_name}', graphData.data, graphData.layout);
    </script>
</body>
</html>
"""

with open(graph_html_path, 'w') as html_file:
    html_file.write(html_content)

## Done ! It's in the website repo, ready for integration and publication.